# MVD 4. cvičení

## 1. část - Načtení dat

Po rozbalení archive.zip uvidíte articles csv soubor. Tento soubor pochází z [Kaggle datasetů](https://www.kaggle.com/hsankesara/medium-articles) a obsahuje malé množství Medium článků k tématům ML, AI a data science. K úloze dnešního cvičení bude stačit využítí dat s názvy a obsahy článků (title a text).


### Příprava dat

Pro přípravu dat se použivá různá sekvence kroků. Je doporučeno na následující kroky vytvořit samostatnou funkci, aby bylo možné zpracovat i vyhledávaný výraz při testování. Dnešní cvičení by mělo obsahovat následující kroky:

1. Převést všechen text na lower case
2. Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
3. Aplikace lemmatizátoru

Pozn.: Jedná se pouze o jednoduchý preprocessing, v praxi je často potřeba použití více kroků. Tato aplikace by měla například problém s čísly (desetinná čísla, čísla vyhledávaná slovně). 

Pro lemmatizaci použijte knihovnu spaCy.

In [162]:
# Instalace spaCy z Jupyter Notebooku
import sys
#!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
#!{sys.executable} -m spacy download en

In [163]:
import spacy
import pandas
import numpy as np

lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # NLTK
# Lemmatizace textu př.:  
# " ".join([token.lemma_ for token in lemmatizer(text)])

C:\Users\podav\anaconda3\envs\data_science\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [164]:
def normalize_data(db):
    # set all to lowercase
    df['text'] = df['text'].str.lower()
    df['title'] = df['title'].str.lower()


    # delete all special characters
    df['text'] = df['text'].str.replace(r'\W', ' ', regex=True)
    df['title'] = df['title'].str.replace(r'\W', ' ', regex=True)
    # delete multiple spaces
    df['text'] = df['text'].str.replace(r'\s+', ' ', regex=True)
    df['title'] = df['title'].str.replace(r'\s+', ' ', regex=True)
    return df

In [165]:
#data load and normalize
df = pandas.read_csv('articles.csv')
df = df[['title','text']]
df_norm = normalize_data(df)
df

,title,text
0,chatbots were the next big thing what happened...,oh how the headlines blared chatbots were the ...
1,python for data science 8 concepts you may hav...,if you ve ever found yourself looking up the s...
2,automated feature engineering in python toward...,machine learning is increasingly moving from h...
3,machine learning how to go from zero to hero f...,if your understanding of a i and machine learn...
4,reinforcement learning from scratch insight data,want to learn about applied artificial intelli...
...,...,...
332,you can build a neural network in javascript e...,click here to share this article on linkedin s...
333,artificial intelligence ai in 2018 and beyond ...,these are my opinions on where deep neural net...
334,spiking neural networks the next generation of...,everyone who has been remotely tuned in to rec...
335,surprise neurons are now more complex than we ...,one of the biggest misconceptions around is th...


## 2. část - Vytvoření invertovaného indexu

Před další prací s textem je potřeba vytvořit invertovaný index, který poté usnadní práci. Invertovaný index bude slovník, kde klíčem bude slovo a hodnotou bude list s id dokumentů (index), které dané slovo obsahují.

Pozn.: Je potřeba vytvořit dva invertované indexy - jeden pro title a druhý pro text.

In [166]:
def inverted_index(texts):
    text_dic = {}
    for i,line in enumerate(texts):
        for word in " ".join([token.lemma_ for token in lemmatizer(line)]).split(' '):
            if word not in text_dic:
                text_dic[word] = [i]
            else:
                text_dic[word].append(i)

    return text_dic

In [167]:
text_ii = inverted_index(df_norm['text'])
title_ii = inverted_index(df_norm['title'])

## 3. část - Implementace TF-IDF

Připravení funkce pro výpočet TF-IDF po příchodu dotazu. Funkce *tf_idf* by měla pracovat s dotazem, jedním invertovaným indexem a s danými dokumenty. Vrátit by měla list obsahující skóre pro každý dokument.

<br>
<center>
$
score(q,d) = TF\_IDF(q,d) = \sum\limits_{w \in q \cap d} c(w, q) c(w, d) log(\frac{M+1}{df(w)})
$
</center>

$q$ ... dotaz<br>
$d$ ... dokument<br>
$c(w, q)$ ... kolikrát je slovo *w* v dotazu *q*<br>
$M$ ... celkový počet dokumentů<br>
$df(w)$ ... počet dokumentů, ve kterých se nachází slovo *w*

In [177]:
def tf(words):
    out = {}
    for word in words:
        if word not in out:
            out[word] = 1
        else:
            out[word] += 1
    return out

def tf_idf(q,d,ii):

    M = len(d) #num of all documents
    q_lem = " ".join([token.lemma_ for token in lemmatizer(q)]).split(' ')
    freq = tf(q_lem) # how many times is each word in query
    out = []
    for doc in d:
        scores = 0
        wd = tf(" ".join([token.lemma_ for token in lemmatizer(doc)]).split(' ')) #how many times is each word in specific document
        for word in q_lem:
            if word in wd:
                scores += freq[word] * wd[word] *  np.log((M+1) / len(set(ii[word])) )
        out.append(scores)
    return  out

In [178]:
score = tf_idf('coursera vs udacity machine learning',df_norm['title'],title_ii)

## 4. část - Použití a testování TF-IDF

Nyní lze získat skóre pro titulky nebo text. Následujícím krokem je sjednocení výsledného skóre pro ohodnocení celého dokumentu. V případě dvou hodnot si vystačíme s parametrem $\alpha$, který nám určuje jakou váhu má titulek a jakou samotný text dokumentu. <br>

<center>
$
score(q,d) = \alpha \; TF\_IDF\_title(q,d) + (1-\alpha) \; TF\_IDF\_text(q,d)
$
</center>

Při nastavení parametru $\alpha$ na hodnotu 0.7 a vyhledávání dotazu "coursera vs udacity machine learning" by výsledky měly vypadat následovně:

![output](sample_output.png)

In [183]:
alpha = 0.7
qs = 'coursera vs udacity machine learning'
test = list(alpha * np.array(tf_idf(qs,df_norm['title'],title_ii) + (1-alpha) * np.array(tf_idf(qs,df_norm['text'],text_ii))))


In [186]:
df_norm['score'] = test
sorted_df = df_norm.sort_values(by='score', ascending=False)
sorted_df

,title,text,score
276,coursera vs udacity for machine learning hacke...,2018 is an exciting time for students of machi...,33.439363
99,every single machine learning course on the in...,a year and a half ago i dropped out of one of ...,23.360618
143,every single machine learning course on the in...,a year and a half ago i dropped out of one of ...,23.360618
19,every single machine learning course on the in...,a year and a half ago i dropped out of one of ...,23.360618
67,every single machine learning course on the in...,a year and a half ago i dropped out of one of ...,23.360618
...,...,...,...
126,how to build a simple neural network in 9 line...,as part of my quest to learn about ai i set my...,0.000000
31,classifying websites with neural networks know...,at datafiniti we have a strong need for conver...,0.000000
298,classifying websites with neural networks know...,at datafiniti we have a strong need for conver...,0.000000
219,sneakpeek the savedroid crypto saving app par...,the international beta of our brand new crypto...,0.000000
